In [14]:
# Rumi Allbert & Daniel Cui

import requests
from bs4 import BeautifulSoup, SoupStrainer
import re
import pandas as pd

# Get HTML from link
mainURL = 'http://en.minghui.org/html/articles/2020/2/10/todays-articles.html'
mainHTML = requests.get(mainURL)
mainHTML.encoding = 'utf-8'
soupMain = BeautifulSoup(mainHTML.text, 'html.parser')
# Filter all links from main links section
linkListHTML = soupMain.findAll('ul', class_='listWrap')
linksHTML = linkListHTML[0].findAll('a')

# Construct lists for titles, links, and word counts.
titleList = []
linkList = []
wordCountList = []

# Fill in titleList, linkList, and wordCountList
for L in linksHTML:
    # Titles
    tx = L.getText().replace('▪', '').strip()
    titleList.append(tx)

    # Links
    linkSubdirectory = L.attrs['href']
    link = 'http://en.minghui.org' + linkSubdirectory
    linkList.append(link)

    # Word Count
    subPageHTML = requests.get(link)
    subPageHTML.encoding = 'utf-8'
    soupSub = BeautifulSoup(subPageHTML.text, 'html.parser')
    bodySub = soupSub.findAll('div', class_='articleZhengwen geo cBBlack')
    paragraphs = bodySub[0].findAll('p')
    '''Make a word counter variable'''
    wordCount = 0
    for paragraph in paragraphs:
        text = paragraph.getText()
        '''remove numbers, special characters, nbsp's, and .org instances'''
        noNum = ''.join(filter(lambda x: not x.isdigit(), text))
        onlyWords = re.sub('[!@#$%^&*(),.?:;\-"/“”+–—\[\]]', ' ', noNum)
        noSpecial = onlyWords.replace('\xa0', ' ').replace(' org ', ' ')
        neatWords = ' '.join(noSpecial.split())
        wordCount += len(neatWords)
    '''Account for the "Minghui" at the beginning of each article'''
    wordCount -= 1
    wordCountList.append(wordCount)

# Construct index list for dataframe
dfIndex = ['Link ' + str(i + 1) for i in range(len(linkList))]

#Create dataframe listing title, link, and word count
linksDF = pd.DataFrame(
    list(zip(titleList, linkList, wordCountList)),
    index=dfIndex,
    columns=['Article Title', 'Article Link', 'Article Word Count'])

#write an excel file using dataframe
writer = pd.ExcelWriter("ExcelProjectFile.xlsx", engine='xlsxwriter')

# Modify column spacing
linksDF.to_excel(writer, sheet_name='Sheet1')
workbook  = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.set_column('B:B', 78)
worksheet.set_column('C:C', 46)
worksheet.set_column('D:D', 17)
writer.save()